In [80]:
import pandas as pd

In [190]:
df = pd.read_csv('../data/curated/impact_news_USD_pair_EURUSD_2007_2018.csv')

In [191]:
df.columns

Index(['actual', 'country', 'datetime', 'forecast', 'forecast_error', 'impact',
       'new', 'previous', 'previous_error', 'week', 'datetime_gmt',
       'forecast_error_ratio', 'year', 'quarter', 'month', 'day_name',
       'open_released', 'high_released', 'low_released', 'close_released',
       'high_5', 'low_5', 'close_5', 'volatility_5', 'direction_5',
       'pips_agg_5', 'pips_candle_5', 'high_10', 'low_10', 'close_10',
       'volatility_10', 'direction_10', 'pips_agg_10', 'pips_candle_10',
       'high_15', 'low_15', 'close_15', 'volatility_15', 'direction_15',
       'pips_agg_15', 'pips_candle_15', 'high_30', 'low_30', 'close_30',
       'volatility_30', 'direction_30', 'pips_agg_30', 'pips_candle_30',
       'high_60', 'low_60', 'close_60', 'volatility_60', 'direction_60',
       'pips_agg_60', 'pips_candle_60', 'high_90', 'low_90', 'close_90',
       'volatility_90', 'direction_90', 'pips_agg_90', 'pips_candle_90',
       'high_120', 'low_120', 'close_120', 'volatility_1

In [206]:
df_2017 = df[(df['year'] == 2017) & (df['impact'] == 'High')]

In [207]:
columns_of_interest = [ 'datetime', 'new', 'impact', 'forecast_error_ratio_zscore', 'total_error_ratio_zscore', 'pips_agg_30', 'pips_agg_60']
df_2017_reduced = df_2017[columns_of_interest]

In [208]:
len(df_2017_reduced)

296

In [209]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df_2017_reduced['new_id'] = lb_make.fit_transform(df_2017_reduced['new'])
df_2017_reduced['new_id'] = df_2017_reduced['new_id'].astype(str)
df_2017_reduced.head() 

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,datetime,new,impact,forecast_error_ratio_zscore,total_error_ratio_zscore,pips_agg_30,pips_agg_60,new_id
1089,2017-01-05 11:00:00,Crude Oil Inventories,High,1.377972,1.377972,13,3,9
1090,2017-01-11 10:30:00,Crude Oil Inventories,High,-1.145583,-1.145583,18,47,9
1091,2017-01-19 11:00:00,Crude Oil Inventories,High,-1.732701,-1.732701,21,20,9
1092,2017-01-25 10:30:00,Crude Oil Inventories,High,0.420723,0.420723,2,9,9
1093,2017-02-01 10:30:00,Crude Oil Inventories,High,0.356983,0.356983,18,12,9


In [210]:
df_2017_by_publication = df_2017_reduced.groupby('datetime').agg({'new_id': lambda x:list(x), \
                                                                  'impact': lambda x:list(x), \
                                                                  'forecast_error_ratio_zscore': lambda x:list(x),\
                                                                  'total_error_ratio_zscore': lambda x:list(x)}).reset_index()
df_2017_by_publication['new_id'] = df_2017_by_publication['new_id'].apply(lambda x: ['_'.join(x)])
df_2017_by_publication.head(5)

,datetime,new_id,impact,forecast_error_ratio_zscore,total_error_ratio_zscore
0,2017-01-03 10:00:00,[12],[High],[0.5446512101847892],[0.5446512101847892]
1,2017-01-05 08:15:00,[0],[High],[-0.584571575480636],[-1.167665126438868]
2,2017-01-05 08:30:00,[20],[High],[1.336276880201799],[1.3740476150525236]
3,2017-01-05 10:00:00,[13],[High],[0.1058103444144838],[0.10581034441448378]
4,2017-01-05 11:00:00,[9],[High],[1.377972006912418],[1.377972006912418]


In [211]:
len(df_2017_by_publication)

218

In [212]:
df_2017_by_publication['new_id'] = df_2017_by_publication['new_id'].apply(lambda x: x[0])
df_2017_by_publication['forecast_error_ratio_zscore'] = df_2017_by_publication['forecast_error_ratio_zscore'].apply(lambda x: sum(x))
df_2017_by_publication['total_error_ratio_zscore'] = df_2017_by_publication['total_error_ratio_zscore'].apply(lambda x: sum(x))
df_2017_by_publication.head(5)

,datetime,new_id,impact,forecast_error_ratio_zscore,total_error_ratio_zscore
0,2017-01-03 10:00:00,12,[High],0.544651,0.544651
1,2017-01-05 08:15:00,0,[High],-0.584572,-1.167665
2,2017-01-05 08:30:00,20,[High],1.336277,1.374048
3,2017-01-05 10:00:00,13,[High],0.105810,0.105810
4,2017-01-05 11:00:00,9,[High],1.377972,1.377972


In [213]:
df_2017_by_publication['num_news'] = df_2017_by_publication['impact'].apply(lambda x: len(x))
df_2017_by_publication_single = df_2017_by_publication[df_2017_by_publication['num_news'] == 1]

In [214]:
df_2017_by_publication_single.head(5)

,datetime,new_id,impact,forecast_error_ratio_zscore,total_error_ratio_zscore,num_news
0,2017-01-03 10:00:00,12,[High],0.544651,0.544651,1
1,2017-01-05 08:15:00,0,[High],-0.584572,-1.167665,1
2,2017-01-05 08:30:00,20,[High],1.336277,1.374048,1
3,2017-01-05 10:00:00,13,[High],0.105810,0.105810,1
4,2017-01-05 11:00:00,9,[High],1.377972,1.377972,1


In [215]:
len(df_2017_by_publication_single)

173

In [216]:
len(df_2017_by_publication)

218

In [225]:
df_2017_reduced = df_2017_reduced.drop(['new_id','new',  'impact', 'forecast_error_ratio_zscore', 'total_error_ratio_zscore'], axis=1)
df_2017_reduced.head(2)

KeyError: "labels ['new_id' 'new' 'impact' 'forecast_error_ratio_zscore'\n 'total_error_ratio_zscore'] not contained in axis"

In [222]:
len(df_2017_reduced)

296

In [223]:
df_2017_reduced = df_2017_reduced.drop_duplicates()

In [226]:
len(df_2017_reduced)

218

In [227]:
df_2017_by_publication = df_2017_by_publication.merge(df_2017_reduced, on='datetime', how= 'left')

df_2017_by_publication.head(4)

,datetime,new_id,impact,forecast_error_ratio_zscore,total_error_ratio_zscore,num_news,pips_agg_30,pips_agg_60
0,2017-01-03 10:00:00,12,[High],0.544651,0.544651,1,26,5
1,2017-01-05 08:15:00,0,[High],-0.584572,-1.167665,1,40,36
2,2017-01-05 08:30:00,20,[High],1.336277,1.374048,1,13,19
3,2017-01-05 10:00:00,13,[High],0.105810,0.105810,1,23,57


In [228]:
len(df_2017_by_publication)

218

In [229]:
df_2017_by_publication.to_csv('../data/curated/groupednews_USD_pair_EURUSD_2007.csv')